<a href="https://colab.research.google.com/github/GabrielWu2004/Numerai/blob/main/XGBoost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numerapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install halo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from numerapi import NumerAPI
from utils_plus import *
import time
import gc

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [13]:
napi = NumerAPI()
spinner = Halo(text='', spinner='dots')
current_round = napi.get_current_round(tournament=8)  

download_data() # train_int8
                # validations_int8
                # features.json


with open("./data/features.json", "r") as f: # use with statement to read files is more convenient
    feature_metadata = json.load(f)
features = feature_metadata["feature_sets"]["medium"]

training_data, validation_data = read_learning_data(features)

X = training_data.filter(like='feature_', axis='columns')
y = training_data[TARGET_COL]

X_test = validation_data.filter(like='feature_', axis='columns')
y_test = validation_data[TARGET_COL]

gc.collect()

Done!
Reading learning data...
Done!


16

In [14]:
print("Feature data type:", type(features)) # list
print("Feature size:", len(features)) # 472
print("feature_metadata type:", type(feature_metadata)) # dictionary
print("keys in feature_metadata:", feature_metadata.keys()) # 'feature_stats', 'feature_sets'
print("keys in feature_sets:", feature_metadata['feature_sets'].keys()) # 'small', 'medium', 'v2_equivalent_features', 'v3_equivalent_features', 'fncv3_features'

Feature data type: <class 'list'>
Feature size: 472
feature_metadata type: <class 'dict'>
keys in feature_metadata: dict_keys(['feature_stats', 'feature_sets'])
keys in feature_sets: dict_keys(['small', 'medium', 'v2_equivalent_features', 'v3_equivalent_features', 'fncv3_features'])


In [16]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(2420521, 472)
(2420521,)
(2421740, 472)
(2421740,)


In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, shuffle=False)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

⠸ Training model(2178468, 472)
(2178468,)
(242053, 472)
(242053,)
⠴ Training model

In [31]:
model_name = "xgboost2"
model_path = ""

⠸ Training model

In [33]:
params = {"n_estimators": 1000,
          "learning_rate": 0.01,
          "max_depth": 10,
          "gamma": 0.5, # pruning
          "reg_lambda": 1, # regularization
          }

model = XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', gpu_id=0, **params)

# training and saving the model
# model name and model paths are defined in earlier cells

if not model_path:
    spinner.start('Training model')
    model.fit(X_train, y_train)
    spinner.succeed()
    gc.collect()
    spinner.start('Saving model')
    model.save_model(f'models/xgboost/{model_name}.txt')
    spinner.succeed()
else:
    model = xgb.Booster(model_file=model_path)
    print(f'Model successfully read from {model_path}\n')

✔ Training model
✔ Saving model


In [36]:
num_feature_neutralization = 50

spinner.start('Predicting on validation data')
validation_data.loc[:, f"preds_{model_name}"] = model.predict(X_test)
spinner.succeed()
gc.collect()

spinner.start('Neutralizing to risky features')
neutralize_riskiest_features(training_data, validation_data, features, model_name, k=num_feature_neutralization)
spinner.succeed()

gc.collect()

print('Exporting Predictions to csv...')
validation_data["prediction"] = validation_data[f"preds_{model_name}_neutral_riskiest_{num_feature_neutralization}"].rank(pct=True)
validation_data["prediction"].to_csv(f"predictions/xgboost/{model_name}.csv")
print('Done!')

✔ Predicting on validation data
✔ Neutralizing to risky features
Exporting Predictions to csv...
Done!
